# ScyllaDB - Data Modeling Best Practices

This notebook demonstrates data modeling patterns and best practices for ScyllaDB.

## Topics Covered
- Query-first design approach
- Partition key selection
- Composite keys and clustering
- Data denormalization
- Counter columns
- Collections (lists, sets, maps)

## Prerequisites
- Understanding of basic ScyllaDB concepts
- Completed previous notebooks

In [1]:
# Import required libraries
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import uuid
from datetime import datetime, timedelta
from decimal import Decimal
import random

## 1. Setup Connection

Connect to ScyllaDB and create our demo keyspace.

In [2]:
# Connect to ScyllaDB
SCYLLA_HOST = "127.0.0.1"
SCYLLA_PORT = 9042

cluster = Cluster([SCYLLA_HOST], port=SCYLLA_PORT)
session = cluster.connect()

print(f"[OK] Connected to ScyllaDB")

# Create keyspace
KEYSPACE = "data_modeling_demo"
session.execute(f"""
    CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
    WITH replication = {{'class': 'SimpleStrategy', 'replication_factor': 1}}
""")
session.set_keyspace(KEYSPACE)
print(f"[OK] Using keyspace '{KEYSPACE}'")

[OK] Connected to ScyllaDB
[OK] Using keyspace 'data_modeling_demo'


## 2. Pattern: Time Series with Bucketing

Partition by time buckets to avoid hot partitions in time series data.

In [3]:
# Create time series table with date bucketing
session.execute("""
    CREATE TABLE IF NOT EXISTS metrics_by_day (
        metric_name text,
        date text,
        timestamp timestamp,
        value double,
        tags map<text, text>,
        PRIMARY KEY ((metric_name, date), timestamp)
    ) WITH CLUSTERING ORDER BY (timestamp DESC)
""")

print("[OK] Created metrics_by_day table")

# Insert sample metrics
metric_names = ["cpu_usage", "memory_usage"]
base_time = datetime.now()

prepared = session.prepare(
    """
        INSERT INTO metrics_by_day (metric_name, date, timestamp, value, tags)
        VALUES (?, ?, ?, ?, ?)
    """
)
for metric in metric_names:
    date_str = base_time.strftime("%Y-%m-%d")
    
    for i in range(10):
        timestamp = base_time - timedelta(minutes=i*5)
        value = random.uniform(0, 100)
        tags = {"host": f"server-{random.randint(1, 3)}"}

        session.execute(prepared, (metric, date_str, timestamp, value, tags))

print(f"[OK] Inserted sample metrics for {len(metric_names)} metrics")

[OK] Created metrics_by_day table
[OK] Inserted sample metrics for 2 metrics


## 3. Counter Columns

Counter columns are perfect for tracking metrics that only increment or decrement.

In [4]:
# Create table with counter columns
session.execute("""
    CREATE TABLE IF NOT EXISTS user_stats (
        user_id uuid PRIMARY KEY,
        followers_count counter,
        following_count counter,
        posts_count counter
    )
""")

print("[OK] Created user_stats table with counters")

# Create a user and update counters
user_id = uuid.uuid4()

# Increment counters
prepared = session.prepare(
"""
    UPDATE user_stats
    SET followers_count = followers_count + 10,
        following_count = following_count + 5,
        posts_count = posts_count + 15
    WHERE user_id = ?
"""
)
session.execute(prepared, (user_id,))

# Query the counters
prepared = session.prepare("SELECT * FROM user_stats WHERE user_id = ?")
result = session.execute(prepared, (user_id,))

print(f"\nUser Statistics for {user_id}:")
for row in result:
    print(f"  Followers: {row.followers_count}")
    print(f"  Following: {row.following_count}")
    print(f"  Posts: {row.posts_count}")

[OK] Created user_stats table with counters

User Statistics for cba49588-03d3-4df5-8a28-3811d4a214fb:
  Followers: 10
  Following: 5
  Posts: 15


## 4. Collections: Lists, Sets, and Maps

Use collections for storing small amounts of related data.

In [5]:
# Create table with collections
session.execute("""
    CREATE TABLE IF NOT EXISTS products (
        product_id uuid PRIMARY KEY,
        name text,
        price decimal,
        categories set<text>,
        tags list<text>,
        attributes map<text, text>
    )
""")

print("[OK] Created products table with collections")

# Insert a product with collections
product_id = uuid.uuid4()
prepared = session.prepare(
"""
    INSERT INTO products (product_id, name, price, categories, tags, attributes)
    VALUES (?, ?, ?, ?, ?, ?)
"""
)
session.execute(
    prepared,
    (
        product_id,
        "Laptop",
        Decimal("999.99"),
        {"electronics", "computers"},
        ["laptop", "portable", "work"],
        {"cpu": "Intel i7", "ram": "16GB", "storage": "512GB SSD"}
    )
)

print("[OK] Inserted product with collections")

# Query and display
prepared = session.prepare("SELECT * FROM products WHERE product_id = ?")
result = session.execute(prepared, (product_id,))

for row in result:
    print(f"\nProduct: {row.name}")
    print(f"  Price: ${row.price}")
    print(f"  Categories: {row.categories}")
    print(f"  Tags: {row.tags}")
    print(f"  Attributes: {row.attributes}")

[OK] Created products table with collections
[OK] Inserted product with collections

Product: Laptop
  Price: $999.99
  Categories: SortedSet(['computers', 'electronics'])
  Tags: ['laptop', 'portable', 'work']
  Attributes: {'cpu': 'Intel i7', 'ram': '16GB', 'storage': '512GB SSD'}


## 5. Updating Collections

Demonstrate adding and removing items from collections.

In [6]:
# Add to set
prepared = session.prepare(
"""
    UPDATE products
    SET categories = categories + {'sale'}
    WHERE product_id = ?
"""
)
session.execute(prepared, (product_id,))

# Append to list
prepared = session.prepare(
"""
    UPDATE products
    SET tags = tags + ['discounted']
    WHERE product_id = ?
"""
)
session.execute(prepared, (product_id,))

# Add to map
prepared = session.prepare(
"""
    UPDATE products
    SET attributes = attributes + {'warranty': '2 years'}
    WHERE product_id = ?
"""
)
session.execute(prepared, (product_id,))

print("[OK] Updated collections")

# Verify updates
prepared = session.prepare("SELECT categories, tags, attributes FROM products WHERE product_id = ?")
result = session.execute(prepared, (product_id,))

for row in result:
    print(f"\nUpdated Collections:")
    print(f"  Categories: {row.categories}")
    print(f"  Tags: {row.tags}")
    print(f"  Attributes: {row.attributes}")

[OK] Updated collections

Updated Collections:
  Categories: SortedSet(['computers', 'electronics', 'sale'])
  Tags: ['laptop', 'portable', 'work', 'discounted']
  Attributes: {'cpu': 'Intel i7', 'ram': '16GB', 'storage': '512GB SSD', 'warranty': '2 years'}


## 6. Data Modeling Best Practices

### Key Principles

**1. Query-First Design**
- Design tables based on your queries, not your entities
- One table per query pattern is often optimal

**2. Partition Key Selection**
- Choose partition keys that distribute data evenly
- Avoid hot partitions
- Consider cardinality and access patterns

**3. Denormalization**
- Duplicate data to avoid joins
- Storage is cheap, queries should be fast
- Accept eventual consistency for better performance

**4. Clustering Columns**
- Use for sorting and range queries
- Order matters for query efficiency
- Can only filter on clustering columns in order

**5. Collections**
- Good for small, related data
- Avoid very large collections (> 64KB recommended max)
- Can't query inside collections efficiently

**6. Counters**
- Perfect for metrics and statistics
- Can't be used with other columns in INSERT
- Eventually consistent

## 7. Anti-Patterns to Avoid

** **Don't:**
- Create "normalized" schemas like in relational databases
- Use ALLOW FILTERING in production queries
- Create very wide rows (> 2GB per partition)
- Store unbounded collections
- Use secondary indexes as primary query method
- Query across multiple partitions regularly

** **Do:**
- Design one table per query pattern
- Use composite partition keys for distribution
- Denormalize data
- Use time bucketing for time series
- Test with production-like data volumes
- Monitor partition sizes

In [7]:
# Cleanup (optional)
# session.execute(f"DROP KEYSPACE IF EXISTS {KEYSPACE}")
# print(f"[OK] Keyspace '{KEYSPACE}' dropped")

cluster.shutdown()
print("\n[OK] Connection closed")


[OK] Connection closed


## Summary

In this notebook, you learned:
- [OK] Query-first design approach
- [OK] Time series data modeling with bucketing
- [OK] Denormalization patterns
- [OK] Using counter columns effectively
- [OK] Collections (sets, lists, maps)
- [OK] Common patterns and anti-patterns

## Resources

- [ScyllaDB Data Modeling Documentation](https://docs.scylladb.com/stable/data-modeling/)
- [ScyllaDB University - Data Modeling Course](https://university.scylladb.com/)
- Previous notebooks in this series

**Congratulations!** You've completed all three ScyllaDB notebooks. You now have a solid understanding of:
1. Basic operations and CRUD
2. Advanced queries and indexing
3. Data modeling best practices

Happy querying! 